## Lenet 5 con MNIST

Imports

In [1]:
! pip install -q tensorflow-model-optimization

In [2]:
import tempfile
import os
import time
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models, losses
import tensorflow_model_optimization as tfmot

Load dataset

In [3]:
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images.shape

(60000, 28, 28)

Normalize the input image so that each pixel value is between 0 to 1.

In [4]:
train_images = tf.pad(train_images, [[0, 0], [2,2], [2,2]])/255
test_images = tf.pad(test_images, [[0, 0], [2,2], [2,2]])/255
train_images.shape

TensorShape([60000, 32, 32])

Most CNN’s accept 4-dimensional tensors as inputs having the dimensions of batch size, height, width, and channel. Since MNIST images are grayscale, the last dimension does not necessarily exist. We need to expand the tensor and create a dummy dimension at axis number 3. (Recall that the tensor initially had axis 0, 1, and 2.)

In [5]:
train_images = tf.expand_dims(train_images, axis=3, name=None)
test_images = tf.expand_dims(test_images, axis=3, name=None)
train_images.shape

TensorShape([60000, 32, 32, 1])

The last 2000 samples of the training set are reserved for the validation set.

In [6]:
val_images = train_images[-2000:,:,:,:]
val_labels = train_labels[-2000:]
train_images = train_images[:-2000,:,:,:]
train_labels = train_labels[:-2000]

Define the model architecture.

In [7]:
model = keras.models.load_model('model.h5')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 6)         156       
                                                                 
 average_pooling2d (AverageP  (None, 14, 14, 6)        0         
 ooling2D)                                                       
                                                                 
 activation (Activation)     (None, 14, 14, 6)         0         
                                                                 
 conv2d_1 (Conv2D)           (None, 10, 10, 16)        2416      
                                                                 
 average_pooling2d_1 (Averag  (None, 5, 5, 16)         0         
 ePooling2D)                                                     
                                                                 
 activation_1 (Activation)   (None, 5, 5, 16)          0

Pruning

In [8]:
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

In [9]:
batch_size=64
epochs=2

num_images = train_images.shape[0]
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

# Define model for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}

model_for_pruning = prune_low_magnitude(model, **pruning_params)

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model_for_pruning.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_conv2d   (None, 28, 28, 6)        308       
 (PruneLowMagnitude)                                             
                                                                 
 prune_low_magnitude_average  (None, 14, 14, 6)        1         
 _pooling2d (PruneLowMagnitu                                     
 de)                                                             
                                                                 
 prune_low_magnitude_activat  (None, 14, 14, 6)        1         
 ion (PruneLowMagnitude)                                         
                                                                 
 prune_low_magnitude_conv2d_  (None, 10, 10, 16)       4818      
 1 (PruneLowMagnitude)                                           
                                                        

Train model

In [12]:
logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]

In [13]:
model_for_pruning.compile(optimizer='adam', loss=losses.sparse_categorical_crossentropy, metrics=['accuracy'])
history = model_for_pruning.fit(train_images, train_labels, batch_size=64, epochs=2, validation_data=(val_images, val_labels))

Epoch 1/2
907/907 [==============================] - 46s 45ms/step - loss: 0.0619 - accuracy: 0.9813 - val_loss: 0.0876 - val_accuracy: 0.9805
Epoch 2/2
907/907 [==============================] - 41s 45ms/step - loss: 0.0643 - accuracy: 0.9811 - val_loss: 0.0625 - val_accuracy: 0.9835


Evaluate

In [15]:
start = time.time()
accuracy = model_for_pruning.evaluate(test_images, test_labels)
end = time.time()
print("Accuracy :",accuracy[1])
print("Images per second :", int(test_images.shape[0]/(end - start)))

313/313 [==============================] - 3s 9ms/step - loss: 0.0636 - accuracy: 0.9808
Accuracy : 0.9807999730110168
Images per second : 1931


In [16]:
# Guardar el Modelo
model_for_pruning.save('model_pruning.h5')
